## **Santa Image Classifier using Transfer Learning on the ImageNet VGG19 Architecture**


In [45]:
import numpy as np
import pandas as pd

In [46]:
!unzip Santas_DataSets.zip

Archive:  Santas_DataSets.zip
replace Santas_DataSets/not_santa/00000000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Santas_DataSets/not_santa/00000000.jpg  
  inflating: Santas_DataSets/not_santa/00000001.jpg  
  inflating: Santas_DataSets/not_santa/00000002.jpg  
  inflating: Santas_DataSets/not_santa/00000003.jpg  
  inflating: Santas_DataSets/not_santa/00000004.jpg  
  inflating: Santas_DataSets/not_santa/00000005.jpg  
  inflating: Santas_DataSets/not_santa/00000006.jpg  
  inflating: Santas_DataSets/not_santa/00000007.jpg  
  inflating: Santas_DataSets/not_santa/00000008.jpg  
  inflating: Santas_DataSets/not_santa/00000009.jpg  
  inflating: Santas_DataSets/not_santa/00000010.jpg  
  inflating: Santas_DataSets/not_santa/00000011.jpg  
  inflating: Santas_DataSets/not_santa/00000012.jpg  
  inflating: Santas_DataSets/not_santa/00000013.jpg  
  inflating: Santas_DataSets/not_santa/00000014.jpg  
  inflating: Santas_DataSets/not_santa/00000015.jpg  
  inflating: Santa

In [47]:
ls

Own_model.h5  santa-claus-orig.jpg  Santas_DataSets.zip
sample_data/  Santas_DataSets/      Tf_Model.h5


In [48]:
cd Santas_DataSets

/content/Santas_DataSets


In [49]:
ls

not_santa/  Own_model.h5  santa/  Tf_Model.h5


In [50]:
import os

In [51]:
not_santa = os.listdir("./not_santa")
santa = os.listdir("./santa")

In [52]:
len(santa)

461

In [53]:
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [54]:
process = imagenet_utils.preprocess_input

In [55]:
Data = []
Classes = []
datanp=[]

In [56]:
for eachsanta in santa:
  try:
    path = "./santa/" + eachsanta
    image = load_img(path, target_size=(224, 224))
    image = img_to_array(image)
    datanp.append(image)
    image = process(image)

    Classes.append("santa")
    Data.append(image)
  except:
    print("Its Not Image")


In [57]:
len(Classes)

461

In [58]:
for eachnotsanta in not_santa:
  try:
    path = "./not_santa/" + eachnotsanta
    image = load_img(path, target_size=(224, 224))
    image = img_to_array(image)
    datanp.append(image)
    image = process(image)

    Classes.append("not_santa")
    Data.append(image)
  except:
    print("Its Not Image")
  

In [59]:
len(Classes)

922

In [60]:
from tensorflow.keras.applications import VGG19
Data = np.array(Data)
datanp=np.array(datanp)##saving the non processed images
Classes = np.array(Classes)
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
Classes = LabelBinarizer().fit_transform(Classes)
NewClasses = OneHotEncoder().fit_transform(Classes).toarray()
trainX, testX, trainY, testY = train_test_split(Data, NewClasses, test_size = 0.2)
baseModel = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))
headModel = baseModel.output
headModel = Flatten()(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
## Add OutPut Layer
headModel = Dense(2, activation="softmax")(headModel)
model = Model(baseModel.input, headModel)
for layer in baseModel.layers:
  layer.trainable = False
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                         height_shift_range=0.1, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")
model.compile(loss="categorical_crossentropy", optimizer="adamax", metrics=["accuracy"])

In [61]:
print("Model Training Start....")
model.fit(aug.flow(trainX, trainY, batch_size=32), epochs=10, validation_data=(testX, testY), verbose=1)

Model Training Start....
Epoch 1/10
24/24 [==============================] - 9s 393ms/step - loss: 3.7650 - accuracy: 0.9023 - val_loss: 1.5546 - val_accuracy: 0.9459
Epoch 2/10
24/24 [==============================] - 9s 386ms/step - loss: 1.3276 - accuracy: 0.9647 - val_loss: 2.3944 - val_accuracy: 0.9351
Epoch 3/10
24/24 [==============================] - 9s 387ms/step - loss: 0.6659 - accuracy: 0.9756 - val_loss: 1.2304 - val_accuracy: 0.9676
Epoch 4/10
24/24 [==============================] - 9s 388ms/step - loss: 0.3551 - accuracy: 0.9756 - val_loss: 1.3691 - val_accuracy: 0.9730
Epoch 5/10
24/24 [==============================] - 9s 387ms/step - loss: 0.3313 - accuracy: 0.9837 - val_loss: 1.2341 - val_accuracy: 0.9730
Epoch 6/10
24/24 [==============================] - 9s 391ms/step - loss: 0.2158 - accuracy: 0.9932 - val_loss: 0.8748 - val_accuracy: 0.9730
Epoch 7/10
24/24 [==============================] - 9s 393ms/step - loss: 0.4087 - accuracy: 0.9837 - val_loss: 0.7991 - va

In [62]:

from PIL import Image
import numpy as np
image = Image.open("/content/santa-claus-orig.jpg")
image = image.resize((224,224))
image = np.expand_dims(image, axis = 0)
image = image/255
pred = np.round(model.predict([image]))
if pred[0][0]==1:
  print("Its Santa")
else:
  print("Not Santa")
   

Its Santa


In [63]:
cd ..

/content


In [64]:
model.save("Tf_Model.h5")

In [65]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential


In [66]:
train1X, test1X, train1Y, test1Y = train_test_split(datanp, NewClasses, test_size = 0.2)

In [67]:
def Create_CNN_Model(h,w,d):
  model1 = Sequential()
  model1.add(Conv2D(32, (3,3), activation="relu", input_shape = (h,w,d)))
  model1.add(BatchNormalization())
  model1.add(MaxPooling2D(pool_size=(2,2)))
  model1.add(Dropout(0.25))


  model1.add(Conv2D(64, (3,3), activation="relu"))
  model1.add(BatchNormalization())
  model1.add(MaxPooling2D(pool_size=(2,2)))
  model1.add(Dropout(0.25))

  model1.add(Conv2D(128, (3,3), activation="relu"))
  model1.add(BatchNormalization())
  model1.add(MaxPooling2D(pool_size=(2,2)))
  model1.add(Dropout(0.25))

  model1.add(Conv2D(128, (3,3), activation="relu"))
  model1.add(BatchNormalization())
  model1.add(MaxPooling2D(pool_size=(2,2)))
  model1.add(Dropout(0.3))

  model1.add(Conv2D(128, (3,3), activation="relu"))
  model1.add(BatchNormalization())
  model1.add(MaxPooling2D(pool_size=(2,2)))
  model1.add(Dropout(0.5))

  model1.add(Flatten())
  model1.add(Dense(512, activation="relu"))
  model1.add(BatchNormalization())
  model1.add(Dropout(0.5))
  


  model1.add(Dense(2, activation="softmax"))
  model1.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  return model

In [68]:
model1 = Create_CNN_Model(224,224,3)

In [69]:
history = model1.fit(train1X,train1Y, epochs=20, validation_data=(test1X,test1Y))

Epoch 1/20
24/24 [==============================] - 5s 223ms/step - loss: 0.5992 - accuracy: 0.9783 - val_loss: 0.2489 - val_accuracy: 0.9892
Epoch 2/20
24/24 [==============================] - 5s 225ms/step - loss: 0.1782 - accuracy: 0.9946 - val_loss: 0.5757 - val_accuracy: 0.9730
Epoch 3/20
24/24 [==============================] - 5s 228ms/step - loss: 0.3313 - accuracy: 0.9864 - val_loss: 0.1658 - val_accuracy: 0.9892
Epoch 4/20
24/24 [==============================] - 5s 226ms/step - loss: 0.0727 - accuracy: 0.9973 - val_loss: 0.1453 - val_accuracy: 0.9892
Epoch 5/20
24/24 [==============================] - 5s 224ms/step - loss: 0.0263 - accuracy: 0.9932 - val_loss: 0.1429 - val_accuracy: 0.9838
Epoch 6/20
24/24 [==============================] - 5s 222ms/step - loss: 0.1272 - accuracy: 0.9932 - val_loss: 0.0552 - val_accuracy: 0.9892
Epoch 7/20
24/24 [==============================] - 5s 220ms/step - loss: 2.3442e-06 - accuracy: 1.0000 - val_loss: 0.0710 - val_accuracy: 0.9784
Ep

In [70]:
model1.save("Own_model.h5")

In [71]:
ls

Own_model.h5  santa-claus-orig.jpg  Santas_DataSets.zip
sample_data/  Santas_DataSets/      Tf_Model.h5
